## Initialize the MemoryHanlder 

In [1]:
import json
from typing import List, Literal, Optional

import tiktoken
from langchain_core.documents import Document
from langchain_core.embeddings import Embeddings
from langchain_core.messages import get_buffer_string
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai.embeddings import OpenAIEmbeddings

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode
import uuid
from MemoryManager import MemoryHandler
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings, NVIDIARerank
import os
import random
choices=["nvidia/llama-3.1-nemotron-51b-instruct","meta/llama-3.1-405b-instruct"]

model=random.choice(choices) #"nvidia/llama-3.1-nemotron-51b-instruct" #"meta/llama-3.1-405b-instruct"
llm = ChatNVIDIA(model=model)
embed = NVIDIAEmbeddings(model="nvidia/nv-embedqa-mistral-7b-v2",truncate="NONE",)
memory_manager=MemoryHandler(llm,embed)


In [2]:
model

'nvidia/llama-3.1-nemotron-51b-instruct'

In [3]:
import os
if os.environ["NVIDIA_API_KEY"]:
    print("exists")

exists


In [4]:
test_d={"a":{"1":1},"b":2}
if "1" in test_d["a"]:
    print(test_d["a"])

{'1': 1}


## wrap into runnable chains with streaming 

In [5]:
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from colorama import Fore

prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "You are assistant with ability to memorize conversations from the user. You should always answer user query based on the following context:\n<Documents>\n{context}\n</Documents>. \
                    Be polite and helpful, make sure your respond sounds natural and remove unnecessary info such as search_memory or quote facts.",
                ),
                ("user", "{input}"),
            ]
        )

memory_retriever_chain = (
    {"context": memory_manager.search_recall_memories, "input": RunnablePassthrough()}
    | prompt
    | llm 
)
   

async def mem_routing_function(inputs):
    query=inputs["input"]
    config=inputs["config"]
    output=await memory_manager.memory_routing(query, config)
    inputs["mem_ops"]=output
    print(Fore.GREEN+"chosen_mem_ops=", output, Fore.RESET)
    return inputs

async def create_memory_items(inputs):
    query=inputs["input"]
    memory_items = await memory_manager.query_to_memory_items(query=query)
    inputs["memory_items"]=memory_items
    return inputs

#runnable_parallel_1 = RunnableLambda(mem_routing_function)
#runnable_parallel_2 = RunnableLambda(create_memory_items)
    

async def execute_memory_operations(inputs):
    mem_ops=inputs["mem_ops"]  
    query=inputs["input"]
    if "search_memory" in mem_ops.lower():
        out= await memory_retriever_chain.ainvoke(query)
        output=out.content
        print(Fore.CYAN+ "integrating response and recall memory items = \n ", output, Fore.RESET)
        memory_items_d= await create_memory_items(inputs)
        memory_items = memory_items_d["memory_items"]["facts"]
        assert type(memory_items)==list 
        #print(Fore.RED +">>>>>>>>>>>>>>>>>>>>> memory_items<<<<<<<<<<<<<<<<<<<<<< \n " , memory_items, Fore.RESET)
        memories, ids= memory_manager.save_recall_memory(memory_items, memory_manager.config)        
    else:        
        user_id=inputs["config"]["configurable"]["user_id"]
        memory_manager.user_id=user_id
        out= await memory_retriever_chain.ainvoke(query)
        output=out.content
        print(Fore.YELLOW + "no memory operation needed continue to respond ", output, Fore.RESET)
    output= output.replace("search_memory","")
    return output

sequence = RunnablePassthrough()  | mem_routing_function  | execute_memory_operations



In [12]:
output=await sequence.ainvoke(input={"input":"hi, my name is Babe, I am a pig and I can talk, my best friend is a chicken named Rob.", "config":config})
output

chosen_mem_ops= search_memory 
integrating response and recall memory items = 
  That's nice to meet you, Babe. However, I can tell that you're quite a conversationalist. Have you and Rob had any sort of disagreement lately? 


"That's nice to meet you, Babe. However, I can tell that you're quite a conversationalist. Have you and Rob had any sort of disagreement lately?"

In [13]:
output

"That's nice to meet you, Babe. However, I can tell that you're quite a conversationalist. Have you and Rob had any sort of disagreement lately?"

## populate the memory with conversations

In [6]:
config = {"configurable": {"user_id": "babe", "thread_id": "1"}}

output=""
async for event in sequence.astream_events(input={"input":"hi, my name is Babe, I am a pig and I can talk, my best friend is a chicken named Rob.", "config":config}):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            output += content
            print(output, end="|", flush=True)


search|search_memory|chosen_mem_ops= search_memory 
search_memoryHello|search_memoryHello Babe|search_memoryHello Babe the|search_memoryHello Babe the talking|search_memoryHello Babe the talking pig|search_memoryHello Babe the talking pig,|search_memoryHello Babe the talking pig, it|search_memoryHello Babe the talking pig, it's|search_memoryHello Babe the talking pig, it's lovely|search_memoryHello Babe the talking pig, it's lovely to|search_memoryHello Babe the talking pig, it's lovely to meet|search_memoryHello Babe the talking pig, it's lovely to meet you|search_memoryHello Babe the talking pig, it's lovely to meet you and|search_memoryHello Babe the talking pig, it's lovely to meet you and your|search_memoryHello Babe the talking pig, it's lovely to meet you and your feather|search_memoryHello Babe the talking pig, it's lovely to meet you and your feathered|search_memoryHello Babe the talking pig, it's lovely to meet you and your feathered friend|search_memoryHello Babe the talking

In [7]:
config = {"configurable": {"user_id": "babe", "thread_id": "2"}}
output=""
async for event in sequence.astream_events(input={"input":"I had a fight with Rob, yes the chicken, he is no longer my best friend, my best friend is now Owen the wise owl!", "config":config}):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            output += content
            print(output, end="|", flush=True)



search|search_memory|chosen_mem_ops= search_memory 
search_memoryI|search_memoryI'm|search_memoryI'm so|search_memoryI'm so sorry|search_memoryI'm so sorry to|search_memoryI'm so sorry to hear|search_memoryI'm so sorry to hear that|search_memoryI'm so sorry to hear that you|search_memoryI'm so sorry to hear that you had|search_memoryI'm so sorry to hear that you had a|search_memoryI'm so sorry to hear that you had a fight|search_memoryI'm so sorry to hear that you had a fight with|search_memoryI'm so sorry to hear that you had a fight with Rob|search_memoryI'm so sorry to hear that you had a fight with Rob the|search_memoryI'm so sorry to hear that you had a fight with Rob the chicken|search_memoryI'm so sorry to hear that you had a fight with Rob the chicken.|search_memoryI'm so sorry to hear that you had a fight with Rob the chicken. It|search_memoryI'm so sorry to hear that you had a fight with Rob the chicken. It's|search_memoryI'm so sorry to hear that you had a fight with Rob the

## integrate into memory normal conversational chain 

In [8]:
config = {"configurable": {"user_id": "babe", "thread_id": "2"}}
output=""
async for event in sequence.astream_events(input={"input":"Do you remember, am i still friends with Rob ?", "config":config}):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            output += content
            print(output, end="|", flush=True)



search|search_memory|chosen_mem_ops= search_memory 
search_memoryActually|search_memoryActually,|search_memoryActually, you|search_memoryActually, you're|search_memoryActually, you're not|search_memoryActually, you're not friends|search_memoryActually, you're not friends with|search_memoryActually, you're not friends with Rob|search_memoryActually, you're not friends with Rob anymore|search_memoryActually, you're not friends with Rob anymore.|search_memoryActually, you're not friends with Rob anymore. There|search_memoryActually, you're not friends with Rob anymore. There was|search_memoryActually, you're not friends with Rob anymore. There was a|search_memoryActually, you're not friends with Rob anymore. There was a fight|search_memoryActually, you're not friends with Rob anymore. There was a fight,|search_memoryActually, you're not friends with Rob anymore. There was a fight, and|search_memoryActually, you're not friends with Rob anymore. There was a fight, and Rob|search_memoryActua

In [9]:
if '{' in output:
    index=output.index('{')
    output=output[:index]
    output=output.replace("search_memory","")
    print(output)

Actually, you're not friends with Rob anymore. There was a fight, and Rob, the chicken, isn't your best friend anymore. Now, Owen the wise owl holds that special place in your life.


In [10]:
def _filter_function(doc: Document) -> bool:
        return True
retriever=memory_manager.recall_vector_store.as_retriever( )
query="Do you remember, am i still friends with Rob ?"
documents = memory_manager.recall_vector_store.search( query=query, search_type='mmr',
         k=10, filter=_filter_function
        )
documents

[Document(id='7f36a16a-2fdc-45fd-afd2-8499e4604c18', metadata={'user_id': 'babe', 'datetime': '2025-08-28'}, page_content='Has a friend named Rob who is a Chicken'),
 Document(id='f737d1bf-a474-420d-8b8b-6862ed56f274', metadata={'user_id': 'babe', 'datetime': '2025-08-28'}, page_content='Can talk'),
 Document(id='111c47f2-b5da-47bf-8229-d6df4342e729', metadata={'user_id': 'babe', 'datetime': '2025-08-28'}, page_content='Had a fight with Rob, the chicken'),
 Document(id='094baef2-78b5-4a00-b36c-6683464fc970', metadata={'user_id': 'babe', 'datetime': '2025-08-28'}, page_content='Rob, the chicken, is no longer their best friend'),
 Document(id='7a35ee87-c16b-4589-9bc4-9382640aa455', metadata={'user_id': 'babe', 'datetime': '2025-08-28'}, page_content='Owen the wise owl is now their best friend'),
 Document(id='11a99a4f-9ba8-4160-9342-43010bb3dc34', metadata={'user_id': 'babe', 'datetime': '2025-08-28'}, page_content='Name is Babe'),
 Document(id='4f58f2b3-5083-4607-97da-99d83963dc4d', met

In [11]:
memory_manager.search_recall_memories(query,config)

['Has a friend named Rob who is a Chicken',
 'Rob, the chicken, is no longer their best friend',
 'Had a fight with Rob, the chicken',
 'Can talk',
 'Is a Pig',
 'Owen the wise owl is now their best friend',
 'Name is Babe']